<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/working_whalepig/Web_Crawling_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
def data_crawling(year, num_top_batter=9):
  # Parameter
  # 원하는 연도 입력
  year = year
  # 데이터를 수집할 타자 중 타석 수 상위 명수
  num_top_batter = 9

  # Team List
  # 원하는 연도에 등록된 팀을 미리 넣어두어야함 (추가 가능)
  Team_2016 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2017 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2018 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2019 = ['NC','kt','LG','두산','키움','KIA','롯데','삼성','SK','한화']
  Team_2020 = ['NC','kt','LG','두산','키움','KIA','롯데','삼성','SK','한화']
  Team_2021 = ['kt','삼성','LG','SSG','NC','키움','두산','롯데','KIA','한화']

  # Team Dictionary
  # Team List를 딕셔너리에 넣음 (추가 가능)
  TeamDic = {'2016':Team_2016, '2017':Team_2017, '2018':Team_2018, '2019':Team_2019, '2020':Team_2020, '2021':Team_2021}

  ###############
  # BeautifulSoup

  import numpy as np
  import pandas as pd
  import requests
  from bs4 import BeautifulSoup
  from tqdm import tqdm

  # 데이터를 모을 dataframe 생성
  data = pd.DataFrame(columns=['name','날짜', '상대', '결과', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실',
                              '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', 'avg', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA'])

  batter_data = pd.DataFrame(columns=['Team','name'])

  for Team in tqdm(TeamDic[str(year)]): # 선택한 연도의 팀을 하나씩 가져옴
    
    # 스탯티즈 시즌기록실 페이지에서 연도, 팀 선택하고, 타석순으로 정렬
    wp = requests.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=" + str(year) + "&ye=" + str(year) + "&se=0&te=" + Team + "&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=TPA&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=")
    soup = BeautifulSoup(wp.text, "html.parser")
    
    # 현재 팀의 타석 수 상위 num_top_batter 명
    tmp_batter_list = np.empty(num_top_batter,dtype=object)
    tmp_team = np.array([Team]*num_top_batter) # 타석 수 상위 타자 팀명 저장
    tmp_batter_list_href = np.empty(num_top_batter,dtype=object)
    tmp_batter_ps=0
    for batter in soup.find_all('tr')[2:11]:
      tmp_batter_list[tmp_batter_ps] = batter.find_all('td')[1].text # 타석 수 상위 타자 이름 저장
      tmp_batter_list_href[tmp_batter_ps] = batter.find_all('a')[0].attrs['href'] # 타석 수 상위 타자 주소 저장
      tmp_batter_ps+=1

    batter_data = pd.concat([batter_data, pd.DataFrame({'Team':tmp_team, 'name':tmp_batter_list})])

    # 타석 수 상위 타자들 중 한명씩 데이터를 모으는 과정
    for top_batter in range(num_top_batter):
      wp=requests.get("http://www.statiz.co.kr/"+tmp_batter_list_href[top_batter]) # 타자 주소로 접근
      soup=BeautifulSoup(wp.text, "html.parser")

      # 날짜별 데이터로 접근
      href_byDate = soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-of-type(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table')[0].find_all('a')[3].attrs['href']
      wp=requests.get("http://www.statiz.co.kr/"+href_byDate)
      soup=BeautifulSoup(wp.text,"html.parser")

      # year : 원하는 연도로 접근
      href_byDate_year=soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-of-type(2) > div > div:nth-of-type(2) > div')[0].find('a', text=str(year)).attrs['href']
      wp=requests.get("http://www.statiz.co.kr/"+href_byDate_year)
      soup=BeautifulSoup(wp.text,"html.parser")

      ## 데이터가 있는 페이지로 이동 완료
      ## 이제 실질적인 데이터를 모을 차례
      rawdata=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-of-type(2) > div > div:nth-of-type(3) > div > div > table')[0]
      # rawdata : 전체 테이블

      # th를 제외한 순수한 경기 데이터 / th는 컬럼명 행이다.
      rawdata_not_th = rawdata.find_all('tr',{'class':['oddrow_stz0','evenrow_stz0']})

      # 행렬만들기
      rawdata_name = np.array([tmp_batter_list[top_batter]]*len(rawdata_not_th))
      rawdata_date = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_vs_team = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_result = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_bat_order = np.zeros(len(rawdata_not_th),dtype=np.int_)
      rawdata_position = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_start_memeber = np.zeros(len(rawdata_not_th), dtype=np.bool)
      rawdata_ab = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_score = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_hit = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_2_hit = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_3_hit = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_homerun = np.zeros(len(rawdata_not_th), dtype=np.int_)
      # 영어명으로 변경 필요
      rawdata_루타 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_타점 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_도루 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_도실 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_볼넷 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_사구 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_고4 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_삼진 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_병살 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_희타 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rewdata_희비 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_avg = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_출루 = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_장타 = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_ops = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_투구 = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_avLI = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_RE24 = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_WPA = np.zeros(len(rawdata_not_th), dtype=np.float)

      for rawdata_not_th_tr_index in range(len(rawdata_not_th)):
        # 한 행 : rawdata_not_th_tr_index
        rawdata_not_th_td = [text for text in rawdata_not_th[rawdata_not_th_tr_index].stripped_strings]
        # 현재 행의 하나의 값 : rawdata_not_th_td[index]
        # 데이터 입력
        rawdata_date[rawdata_not_th_tr_index] = rawdata_not_th_td[1]
        rawdata_vs_team[rawdata_not_th_tr_index] = rawdata_not_th_td[2]
        rawdata_result[rawdata_not_th_tr_index] = rawdata_not_th_td[3]+" "+rawdata_not_th_td[4]
        rawdata_bat_order[rawdata_not_th_tr_index] = rawdata_not_th_td[5]
        rawdata_position[rawdata_not_th_tr_index] = rawdata_not_th_td[6]
        rawdata_start_memeber[rawdata_not_th_tr_index] = rawdata_not_th_td[7]
        rawdata_ab[rawdata_not_th_tr_index] = rawdata_not_th_td[8]
        rawdata_score[rawdata_not_th_tr_index] = rawdata_not_th_td[9]
        rawdata_hit[rawdata_not_th_tr_index] = rawdata_not_th_td[10]
        rawdata_2_hit[rawdata_not_th_tr_index] = rawdata_not_th_td[11]
        rawdata_3_hit[rawdata_not_th_tr_index] = rawdata_not_th_td[12]
        rawdata_homerun[rawdata_not_th_tr_index] = rawdata_not_th_td[13]
        rawdata_루타[rawdata_not_th_tr_index] = rawdata_not_th_td[14]
        rawdata_타점[rawdata_not_th_tr_index] = rawdata_not_th_td[15]
        rawdata_도루[rawdata_not_th_tr_index] = rawdata_not_th_td[16]
        rawdata_도실[rawdata_not_th_tr_index] = rawdata_not_th_td[17]
        rawdata_볼넷[rawdata_not_th_tr_index] = rawdata_not_th_td[18]
        rawdata_사구[rawdata_not_th_tr_index] = rawdata_not_th_td[19]
        rawdata_고4[rawdata_not_th_tr_index] = rawdata_not_th_td[20]
        rawdata_삼진[rawdata_not_th_tr_index] = rawdata_not_th_td[21]
        rawdata_병살[rawdata_not_th_tr_index] = rawdata_not_th_td[22]
        rawdata_희타[rawdata_not_th_tr_index] = rawdata_not_th_td[23]
        rewdata_희비[rawdata_not_th_tr_index] = rawdata_not_th_td[24]
        rawdata_avg[rawdata_not_th_tr_index] = rawdata_not_th_td[25]
        rawdata_출루[rawdata_not_th_tr_index] = rawdata_not_th_td[26]
        rawdata_장타[rawdata_not_th_tr_index] = rawdata_not_th_td[27]
        rawdata_ops[rawdata_not_th_tr_index] = rawdata_not_th_td[28]
        rawdata_투구[rawdata_not_th_tr_index] = rawdata_not_th_td[29]
        rawdata_avLI[rawdata_not_th_tr_index] = rawdata_not_th_td[30]
        rawdata_RE24[rawdata_not_th_tr_index] = rawdata_not_th_td[31]
        rawdata_WPA[rawdata_not_th_tr_index] = rawdata_not_th_td[32]

      tmp_data = pd.DataFrame()
      # tmp_data는 각 선수마다 데이터를 모은 후 data에 머지하기 전에 존재
      tmp_data['name'] = rawdata_name
      tmp_data['날짜'] = rawdata_date
      tmp_data['상대'] = rawdata_vs_team
      tmp_data['결과'] = rawdata_result
      tmp_data['타순'] = rawdata_bat_order
      tmp_data['P'] = rawdata_position
      tmp_data['선발'] = rawdata_start_memeber
      tmp_data['타수'] = rawdata_ab
      tmp_data['득점'] = rawdata_score
      tmp_data['안타'] = rawdata_hit
      tmp_data['2타'] = rawdata_2_hit
      tmp_data['3타'] = rawdata_3_hit
      tmp_data['홈런'] = rawdata_homerun
      tmp_data['루타'] = rawdata_루타
      tmp_data['타점'] = rawdata_타점
      tmp_data['도루'] = rawdata_도루
      tmp_data['도실'] = rawdata_도실
      tmp_data['볼넷'] = rawdata_볼넷
      tmp_data['사구'] = rawdata_사구
      tmp_data['고4'] = rawdata_고4
      tmp_data['삼진'] = rawdata_삼진
      tmp_data['병살'] = rawdata_병살
      tmp_data['희타'] = rawdata_희타
      tmp_data['희비'] = rewdata_희비
      tmp_data['avg'] = rawdata_avg
      tmp_data['출루'] = rawdata_출루
      tmp_data['장타'] = rawdata_장타
      tmp_data['OPS'] = rawdata_ops
      tmp_data['투구'] = rawdata_투구
      tmp_data['avLI'] = rawdata_avLI
      tmp_data['RE24'] = rawdata_RE24
      tmp_data['WPA'] = rawdata_WPA

      # 데이터 쌓기
      data = pd.concat([data,tmp_data])

  return data, batter_data


In [30]:
data_2018, batter_data_2018 = data_crawling(2018,9)
data_2019, batter_data_2019 = data_crawling(2019,9)
data_2020, batter_data_2020 = data_crawling(2020,9)
data_2021, batter_data_2021 = data_crawling(2021,9)


100%|██████████| 10/10 [10:23<00:00, 62.33s/it]

100%|██████████| 10/10 [07:36<00:00, 45.61s/it]

100%|██████████| 10/10 [07:57<00:00, 47.73s/it]

100%|██████████| 10/10 [09:03<00:00, 54.35s/it]


In [42]:
data_2018.to_csv("data_2018.csv", encoding='euc-kr', index=False)
data_2019.to_csv("data_2019.csv", encoding='euc-kr', index=False)
data_2020.to_csv("data_2020.csv", encoding='euc-kr', index=False)
data_2021.to_csv("data_2021.csv", encoding='euc-kr', index=False)

batter_data_2018.to_csv("batter_data_2018.csv", encoding='euc-kr', index=False)
batter_data_2019.to_csv("batter_data_2019.csv", encoding='euc-kr', index=False)
batter_data_2020.to_csv("batter_data_2020.csv", encoding='euc-kr', index=False)
batter_data_2021.to_csv("batter_data_2021.csv", encoding='euc-kr', index=False)